In [0]:
!pip install vecstack

from vecstack import stacking
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR

from sklearn.linear_model import LogisticRegression
#add following to your code
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
trainfile = r'/gdrive/My Drive/SCM516/train.csv'
P_trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/SCM516/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

trainData=P_trainData.drop(columns=['Survived'])
print(trainData.shape)

print(testData.shape)
trainData.head()


(891, 11)
(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
testData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
Processing = combined_Data.dropna(thresh=percent,axis=1)
print(Processing.shape)


785
(1309, 10)


In [0]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric = Processing.select_dtypes(include=['int','float64']).columns
for num in numeric:
  Processing[num] = Processing[num].fillna(Processing[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = Processing.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  Processing[colss] = Processing[colss].fillna(Processing[colss].mode()[0])

print(Processing.head())



     PassengerId  Pclass                                               Name     Sex   Age  SibSp  Parch            Ticket     Fare Embarked
0 0            1       3                            Braund, Mr. Owen Harris    male  22.0      1      0         A/5 21171   7.2500        S
  1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1      0          PC 17599  71.2833        C
  2            3       3                             Heikkinen, Miss. Laina  female  26.0      0      0  STON/O2. 3101282   7.9250        S
  3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1      0            113803  53.1000        S
  4            5       3                           Allen, Mr. William Henry    male  35.0      0      0            373450   8.0500        S


In [0]:
#Check final state of nulls in the columns
Processing.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [0]:
#dropping unnecessary columns
Processing_F=Processing.drop(columns=['Name','Ticket'])
print(Processing_F.shape)

(1309, 8)


In [0]:
#Export Processed data 
F_Train = Processing_F.xs(0)
F_Test = Processing_F.xs(1)

#
combine = [F_Train, F_Test]
F_Train.shape, F_Test.shape
for dataset in combine:
  dataset['Sex'] = dataset['Sex'].map({'female': 1, 'male': 0}).astype(int)


F_Train.to_csv(r'/gdrive/My Drive/SCM516/Preprocess_Train.csv',index=None)
F_Test.to_csv(r'/gdrive/My Drive/SCM516/Preprocess_Test.csv',index=None)

In [0]:
F_Train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,S
1,2,1,1,38.0,1,0,71.2833,C
2,3,3,1,26.0,0,0,7.9250,S
3,4,1,1,35.0,1,0,53.1000,S
4,5,3,0,35.0,0,0,8.0500,S


In [0]:
F_Test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,Q
1,893,3,1,47.0,1,0,7.0000,S
2,894,2,0,62.0,0,0,9.6875,Q
3,895,3,0,27.0,0,0,8.6625,S
4,896,3,1,22.0,1,1,12.2875,S


In [0]:
#Import Processed data and check it
trainfile = r'/gdrive/My Drive/SCM516/Preprocess_Train.csv'
trainData = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/SCM516/Preprocess_Test.csv'
testData = pd.read_csv(testfile)

print(trainData.shape)
print(testData.shape)


(891, 8)
(418, 8)


In [0]:
trainData.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,0.352413,29.735277,0.523008,0.381594,32.204208
std,257.353842,0.836071,0.477990,13.002218,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,0.000000,29.881138,0.000000,0.000000,14.454200
75%,668.500000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [0]:
trainData.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,S
1,2,1,1,38.0,1,0,71.2833,C
2,3,3,1,26.0,0,0,7.9250,S
3,4,1,1,35.0,1,0,53.1000,S
4,5,3,0,35.0,0,0,8.0500,S


In [0]:
testData.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,Q
1,893,3,1,47.0,1,0,7.0000,S
2,894,2,0,62.0,0,0,9.6875,Q
3,895,3,0,27.0,0,0,8.6625,S
4,896,3,1,22.0,1,1,12.2875,S


In [0]:
#Copy Train data excluding ID column
trainData_Copy = trainData.iloc[:,1:].copy()
#Copy Test data excluding ID column 
testData_Copy = testData.iloc[:,1:].copy()

print(trainData_Copy.head())
print(testData_Copy.head())

   Pclass  Sex   Age  SibSp  Parch     Fare Embarked
0       3    0  22.0      1      0   7.2500        S
1       1    1  38.0      1      0  71.2833        C
2       3    1  26.0      0      0   7.9250        S
3       1    1  35.0      1      0  53.1000        S
4       3    0  35.0      0      0   8.0500        S
   Pclass  Sex   Age  SibSp  Parch     Fare Embarked
0       3    0  34.5      0      0   7.8292        Q
1       3    1  47.0      1      0   7.0000        S
2       2    0  62.0      0      0   9.6875        Q
3       3    0  27.0      0      0   8.6625        S
4       3    1  22.0      1      1  12.2875        S


In [0]:
#List of Categorical Features
#categoricalFeatures = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
train_cat_cols = trainData_Copy.select_dtypes(exclude=['float','int']).columns
print(train_cat_cols.shape)
print(train_cat_cols)

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy, testData_Copy], keys=[2,3])
print(combined_Data.shape)

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=train_cat_cols)
print(combined_Data.shape)

#save feature names because we are going to do rescaler
combined_Data_col = combined_Data.columns

#Separate Train data and test data
X_Train = combined_Data.xs(2)
X_Test = combined_Data.xs(3)

#Normalize for train data
Xscaler = StandardScaler()
Xscaled = Xscaler.fit(X_Train)
XscalerTrans = Xscaler.transform(X_Train)
#give feature names back which we saved before
X_Train = pd.DataFrame(XscalerTrans, columns=combined_Data_col)
print(X_Train.head())
print(X_Train.shape)

#Normalize for test data
Xscaled = Xscaler.fit(X_Test)
XscalerTrans = Xscaler.transform(X_Test)
#give feature names back which we saved before
X_Test = pd.DataFrame(XscalerTrans, columns=combined_Data_col)


(1,)
Index(['Embarked'], dtype='object')
(1309, 7)
(1309, 9)
     Pclass       Sex       Age     SibSp     Parch      Fare  Embarked_C  Embarked_Q  Embarked_S
0  0.827377 -0.737695 -0.595254  0.432793 -0.473674 -0.502445   -0.482043   -0.307562    0.615838
1 -1.566107  1.355574  0.635996  0.432793 -0.473674  0.786845    2.074505   -0.307562   -1.623803
2  0.827377  1.355574 -0.287441 -0.474545 -0.473674 -0.488854   -0.482043   -0.307562    0.615838
3 -1.566107  1.355574  0.405137  0.432793 -0.473674  0.420730   -0.482043   -0.307562    0.615838
4  0.827377 -0.737695  0.405137 -0.474545 -0.473674 -0.486337   -0.482043   -0.307562    0.615838
(891, 9)


In [0]:
#X_Test.head()
print(X_Train.shape)
print(X_Test.shape)

print(X_Train.head)
print(X_Test.head)

(891, 9)
(418, 9)
<bound method NDFrame.head of        Pclass       Sex       Age     SibSp     Parch      Fare  Embarked_C  Embarked_Q  Embarked_S
0    0.827377 -0.737695 -0.595254  0.432793 -0.473674 -0.502445   -0.482043   -0.307562    0.615838
1   -1.566107  1.355574  0.635996  0.432793 -0.473674  0.786845    2.074505   -0.307562   -1.623803
2    0.827377  1.355574 -0.287441 -0.474545 -0.473674 -0.488854   -0.482043   -0.307562    0.615838
3   -1.566107  1.355574  0.405137  0.432793 -0.473674  0.420730   -0.482043   -0.307562    0.615838
4    0.827377 -0.737695  0.405137 -0.474545 -0.473674 -0.486337   -0.482043   -0.307562    0.615838
..        ...       ...       ...       ...       ...       ...         ...         ...         ...
886 -0.369365 -0.737695 -0.210488 -0.474545 -0.473674 -0.386671   -0.482043   -0.307562    0.615838
887 -1.566107  1.355574 -0.826113 -0.474545 -0.473674 -0.044381   -0.482043   -0.307562    0.615838
888  0.827377  1.355574  0.011224  0.432793  2.00893

In [0]:
#Select Target column
Y_Train = P_trainData["Survived"]
print(Y_Train.shape)
print(Y_Train.head())

(891,)
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [0]:
#Logistic Regressor ========================================================================
#CONSTRUCT DEFAULT Logistic AND OBTAIN RESPECTIVE ACCURACY 
lr = LogisticRegression()
lr.fit(X_Train, Y_Train)
lr_predict_train=lr.predict(X_Train)

#lr.feature_importances_
mean_squared_error(Y_Train,lr_predict_train)
print("RMSE (training) for Logistic Regression:{0:10f}".format(mean_squared_error(Y_Train,lr_predict_train)))

#test data
lr_predict_Test=lr.predict(X_Test)

pred=pd.DataFrame(lr_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_LR.csv',index=None)

RMSE (training) for Logistic Regression:  0.199776
   Survived
0         0
1         0
2         0
3         0
4         1
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1


In [0]:
#Hyperparameter tuning done for Logistic Regression 
parameters={       
                   'max_iter': range(10,100,10)
                   
            }
lr_random = RandomizedSearchCV(lr,parameters,n_iter=10)
lr_random.fit(X_Train,Y_Train)
rand_parm=lr_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTree 
lr = LogisticRegression(**rand_parm)
lr.fit(X_Train,Y_Train)
lr_predict_train = lr.predict(X_Train)

#lr.feature_importances_
mean_squared_error(Y_Train,lr_predict_train)
print("RMSE (training) for Logistic Regression :{0:10f}".format(mean_squared_error(Y_Train,lr_predict_train)))

#run cross-validation on best hyperparameters
lr_cv_score = cross_val_score(lr, X_Train, Y_Train, cv=8)

print(lr_cv_score)
print('\n')

print(" - Logistic Regression : ",lr_cv_score.mean())

#test data
lr_predict_Test=lr.predict(X_Test)

pred=pd.DataFrame(lr_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_LR.csv',index=None)

{'max_iter': 10}
RMSE (training) for Logistic Regression :  0.199776
[0.78571429 0.79464286 0.80357143 0.80357143 0.77678571 0.76576577
 0.79090909 0.84545455]


 - Logistic Regression :  0.7958018895518895
   Survived
0         0
1         0
2         0
3         0
4         1
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1


In [0]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor()
clf.fit(X_Train, Y_Train)
clf_predict_train=clf.predict(X_Train)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#test data
clf_predict_Test=clf.predict(X_Test)

pred=pd.DataFrame(clf_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_DT.csv',index=None)




RMSE (training) for Decision Tree:  0.010784
   Survived
0       0.0
1       0.0
2       1.0
3       0.0
4       1.0
   PassengerId  Survived
0          892       0.0
1          893       0.0
2          894       1.0
3          895       0.0
4          896       1.0


In [0]:
#Hyperparameter tuning done for decision tree 
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(X_Train,Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTree 
clf = DecisionTreeRegressor(**rand_parm)
clf.fit(X_Train,Y_Train)
clf_predict_train = clf.predict(X_Train)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#run cross-validation on best hyperparameters
clf_cv_score = cross_val_score(clf, X_Train, Y_Train, cv=8)

print(clf_cv_score)
print('\n')

print(" - Decision Tree: ",clf_cv_score.mean())

#test data
clf_predict_Test=clf.predict(X_Test)

pred=pd.DataFrame(clf_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_DT.csv',index=None)

{'min_samples_split': 80, 'min_samples_leaf': 3}
RMSE (training) for Decision Tree:  0.116257
[0.20142189 0.40657743 0.31489669 0.53888997 0.3385994  0.42541648
 0.49533713 0.51911202]


 - Decision Tree:  0.40503137576926107
   Survived
0  0.000000
1  0.142857
2  0.142857
3  0.000000
4  0.526316
   PassengerId  Survived
0          892  0.000000
1          893  0.142857
2          894  0.142857
3          895  0.000000
4          896  0.526316


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestRegressor()
rfc.fit(X_Train,Y_Train)
rfc_predict_train=rfc.predict(X_Train)

mean_squared_error(Y_Train,rfc_predict_train)
print("RMSE (training) for random forest :{0:10f}".format(mean_squared_error(Y_Train,rfc_predict_train)))


#test data
rfc_predict_Test=rfc.predict(X_Test)

pred=pd.DataFrame(rfc_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_RF.csv',index=None)


RMSE (training) for random forest :  0.033623
   Survived
0       0.0
1       0.2
2       0.3
3       0.4
4       0.6
   PassengerId  Survived
0          892       0.0
1          893       0.2
2          894       0.3
3          895       0.4
4          896       0.6


In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(X_Train,Y_Train)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestRegressor(**rand_parm_rfc)
rfc.fit(X_Train,Y_Train)
rfc_predict_train = rfc.predict(X_Train)


mean_squared_error(Y_Train,rfc_predict_train)
print("RMSE (training) for random forest:{0:10f}".format(mean_squared_error(Y_Train,rfc_predict_train)))

#run cross-validation on best hyperparameters
rfc_cv_score = cross_val_score(rfc, X_Train, Y_Train, cv=8)

print(rfc_cv_score)
print('\n')

print(" - rf: ",rfc_cv_score.mean())

#test data
rfc_predict_Test=rfc.predict(X_Test)

pred=pd.DataFrame(rfc_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_RF.csv',index=None)

{'n_estimators': 60, 'min_samples_split': 20, 'min_samples_leaf': 5}
RMSE (training) for random forest:  0.093715
[0.25785475 0.36455364 0.39262969 0.54164463 0.43526499 0.43591874
 0.51262745 0.51511137]


 - rf:  0.4319506585464633
   Survived
0  0.017437
1  0.442581
2  0.113238
3  0.096128
4  0.531576
   PassengerId  Survived
0          892  0.017437
1          893  0.442581
2          894  0.113238
3          895  0.096128
4          896  0.531576


In [0]:
#gradient boosting default

grd =GradientBoostingRegressor()
grd.fit(X_Train, Y_Train)
grd_predict_train=grd.predict(X_Train)


mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#test data
grd_predict_Test=grd.predict(X_Test)

pred=pd.DataFrame(grd_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_GB.csv',index=None)#

RMSE (training) for gradient bossting:  0.089539
   Survived
0  0.046177
1  0.204234
2  0.246388
3  0.130970
4  0.452216
   PassengerId  Survived
0          892  0.046177
1          893  0.204234
2          894  0.246388
3          895  0.130970
4          896  0.452216


In [0]:
#Hyperparameter tuning for gb

parameters={ 'max_depth': [100,150],'learning_rate': [0.1,0.2]}
grd_random = RandomizedSearchCV(grd,parameters,n_iter=10)
grd_random.fit(X_Train, Y_Train)
rand_parm_grd=grd_random.best_params_
print(rand_parm_grd)

#contruct gb using the best parameters
grd= GradientBoostingRegressor(**rand_parm_grd)
grd.fit(X_Train, Y_Train)
grd_predict_train=grd.predict(X_Train)

mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#run cross-validation on best hyperparameters
grd_cv_score = cross_val_score(grd, X_Train, Y_Train, cv=8)

print(grd_cv_score)
print('\n')

print(" - gb: ",grd_cv_score.mean())

#test data
grd_predict_Test=grd.predict(X_Test)

pred=pd.DataFrame(grd_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_GB.csv',index=None)

{'max_depth': 150, 'learning_rate': 0.1}
RMSE (training) for gradient bossting:  0.010784
[-0.09549864 -0.0803957   0.13060881  0.23699689  0.14692709  0.30111963
  0.04436962  0.21547154]


 - gb:  0.11244990541865692
   Survived
0  0.000008
1  0.000010
2  0.999728
3  0.000063
4  0.999727
   PassengerId  Survived
0          892  0.000008
1          893  0.000010
2          894  0.999728
3          895  0.000063
4          896  0.999727


In [0]:
#MLP Regressor================================================================================
#default
nn =MLPRegressor()
nn.fit(X_Train, Y_Train)
nn_predict_train=nn.predict(X_Train)

mean_squared_error(Y_Train,nn_predict_train)
print("RMSE (training) for nn:{0:10f}".format(mean_squared_error(Y_Train,nn_predict_train)))

#test data
nn_predict_Test=nn.predict(X_Test)

pred=pd.DataFrame(nn_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_NN.csv',index=None)#

RMSE (training) for nn:  0.103979
   Survived
0  0.049788
1  0.438861
2 -0.120882
3  0.107072
4  0.465263
   PassengerId  Survived
0          892  0.049788
1          893  0.438861
2          894 -0.120882
3          895  0.107072
4          896  0.465263


In [0]:
#Hyperparameter tuning for mlp
parameters={ 'learning_rate': ['invscaling', 'adaptive'], 'solver': ['lbfgs', 'sgd', 'adam']}
#
nn_random = RandomizedSearchCV(nn,parameters,n_iter=10)
nn_random.fit(X_Train, Y_Train)
rand_parm_nn=nn_random.best_params_
print(rand_parm_nn)

#contruct mlp using the best parameters
nn= MLPRegressor(**rand_parm_nn)
nn.fit(X_Train,Y_Train)
nn_predict_train=nn.predict(X_Train)

mean_squared_error(Y_Train,nn_predict_train)
print("RMSE (training) for nn:{0:10f}".format(mean_squared_error(Y_Train,nn_predict_train)))

#run cross-validation on best hyperparameters
nn_cv_score = cross_val_score(nn, X_Train, Y_Train, cv=8)

print(nn_cv_score)
print('\n')

print(" - nn: ",nn_cv_score.mean())

#test data
nn_predict_Test=nn.predict(X_Test)

pred=pd.DataFrame(nn_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_NN.csv',index=None)#

{'solver': 'adam', 'learning_rate': 'adaptive'}
RMSE (training) for nn:  0.109577
[0.37154584 0.39624587 0.38852869 0.47842854 0.4449839  0.37671076
 0.44811468 0.50944322]


 - nn:  0.4267501880333209
   Survived
0 -0.008097
1  0.416054
2  0.000781
3  0.159748
4  0.395683
   PassengerId  Survived
0          892 -0.008097
1          893  0.416054
2          894  0.000781
3          895  0.159748
4          896  0.395683


In [0]:
#LinearSVR================================================================================

svr =LinearSVR()
svr.fit(X_Train, Y_Train)
svr_predict_train=svr.predict(X_Train)

mean_squared_error(Y_Train,svr_predict_train)
print("RMSE (training) for svr:{0:10f}".format(mean_squared_error(Y_Train,svr_predict_train)))


#test data
svr_predict_Test=svr.predict(X_Test)

pred=pd.DataFrame(svr_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_SVR.csv',index=None)#

RMSE (training) for svr:  0.213244
   Survived
0 -0.008711
1  0.984380
2 -0.008711
3 -0.008711
4  0.984380
   PassengerId  Survived
0          892 -0.008711
1          893  0.984380
2          894 -0.008711
3          895 -0.008711
4          896  0.984380


In [0]:
#Hyperparameter tuning for svr
parameters={ 'loss': ['epsilon_insensitive','squared_epsilon_insensitive'], 'max_iter': [1000,2000]}
svr_random = RandomizedSearchCV(svr,parameters,n_iter=10)
svr_random.fit(X_Train, Y_Train)
rand_parm_svr=svr_random.best_params_
print(rand_parm_svr)

#contruct svr using the best parameters
svr= LinearSVR(**rand_parm_svr)
svr.fit(X_Train,Y_Train)
svr_predict_train=svr.predict(X_Train)

mean_squared_error(Y_Train,svr_predict_train)
print("RMSE (training) for svr:{0:10f}".format(mean_squared_error(Y_Train,svr_predict_train)))

#run cross-validation on best hyperparameters
svr_cv_score = cross_val_score(svr, X_Train, Y_Train, cv=8)

print(svr_cv_score)
print('\n')

print(" - svr: ",svr_cv_score.mean())

#test data
svr_predict_Test=svr.predict(X_Test)

pred=pd.DataFrame(svr_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_SVR.csv',index=None)

{'max_iter': 2000, 'loss': 'squared_epsilon_insensitive'}
RMSE (training) for svr:  0.142322
[0.38398302 0.31432662 0.37070325 0.38294329 0.37415868 0.30901744
 0.45233053 0.41933958]


 - svr:  0.3758503013524174
   Survived
0  0.108149
1  0.424430
2  0.113691
3  0.095057
4  0.563262
   PassengerId  Survived
0          892  0.108149
1          893  0.424430
2          894  0.113691
3          895  0.095057
4          896  0.563262


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using Neural networks, RandomForest and Decision Tree Classifier\n")

models = [MLPRegressor(),  RandomForestRegressor(), DecisionTreeRegressor() ]#we r giving models as arguments we can also give those parameters like grid_params
      #we are giving  data after smote and we did smote only for train test
Strain, Stest = stacking(models,                   
                           X_Train, Y_Train, X_Test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                         
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
                         
                         verbose=2)
         

___________________________________________________________________________________________
Ensemble Methods Predictions using Neural networks, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPRegressor]
    fold  0:  [0.27847019]
    fold  1:  [0.26780124]
    fold  2:  [0.26262430]
    fold  3:  [0.27044670]
    ----
    MEAN:     [0.26983560] + [0.00572431]
    FULL:     [0.26983492]

model  1:     [RandomForestRegressor]
    fold  0:  [0.24292832]
    fold  1:  [0.22821623]
    fold  2:  [0.24792174]
    fold  3:  [0.23949378]
    ----
    MEAN:     [0.23964002] + [0.00724438]
    FULL:     [0.23964018]

model  2:     [DecisionTreeRegressor]
    fold  0:  [0.25071001]
    fold  1:  [0.24140508]
    fold  2:  [0.23657912]
    fold  3:  [0.23198198]
    ----
    MEAN:     [0.24016905] + [0.00693822]
    FULL:     [0.24017824]



In [0]:
#stacking one layer dt
clf = DecisionTreeRegressor()
clf.fit(Strain, Y_Train)
clf_predict_train=clf.predict(Strain)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Stacking Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#test data
clf_predict_Test=clf.predict(Stest)

pred=pd.DataFrame(clf_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_DT_Stack.csv',index=None)

RMSE (training) for Stacking Decision Tree:  0.003835
   Survived
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
   PassengerId  Survived
0          892       0.0
1          893       0.0
2          894       0.0
3          895       0.0
4          896       0.0


In [0]:
#Hyperparameter tuning done for decision tree 
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(Strain,Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTree 
clf = DecisionTreeRegressor(**rand_parm)
clf.fit(Strain,Y_Train)
clf_predict_train = clf.predict(Strain)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#run cross-validation on best hyperparameters
clf_cv_score = cross_val_score(clf, Strain, Y_Train, cv=8)

print(clf_cv_score)
print('\n')

print(" - Decision Tree: ",clf_cv_score.mean())

#test data
clf_predict_Test=clf.predict(Stest)

pred=pd.DataFrame(clf_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_DT_Stack.csv',index=None)

{'min_samples_split': 70, 'min_samples_leaf': 7}
RMSE (training) for Decision Tree:  0.114069
[0.30590156 0.28160093 0.33984721 0.47451561 0.43402266 0.48321198
 0.43193638 0.39967568]


 - Decision Tree:  0.393839000311502
   Survived
0  0.029412
1  0.103448
2  0.029412
3  0.166667
4  0.592593
   PassengerId  Survived
0          892  0.029412
1          893  0.103448
2          894  0.029412
3          895  0.166667
4          896  0.592593


In [0]:
#gradient boosting default

grd =GradientBoostingRegressor()
grd.fit(Strain, Y_Train)
grd_predict_train=grd.predict(Strain)


mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#test data
grd_predict_Test=grd.predict(Stest)

pred=pd.DataFrame(grd_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_DF_GB_Stack.csv',index=None)#

RMSE (training) for gradient bossting:  0.085611
   Survived
0  0.017066
1  0.109169
2  0.647276
3  0.074094
4  0.533378
   PassengerId  Survived
0          892  0.017066
1          893  0.109169
2          894  0.647276
3          895  0.074094
4          896  0.533378


In [0]:
#Hyperparameter tuning for gb

parameters={ 'max_depth': [100,150],'learning_rate': [0.1,0.2]}
grd_random = RandomizedSearchCV(grd,parameters,n_iter=10)
grd_random.fit(Strain, Y_Train)
rand_parm_grd=grd_random.best_params_
print(rand_parm_grd)

#contruct gb using the best parameters
grd= GradientBoostingRegressor(**rand_parm_grd)
grd.fit(Strain, Y_Train)
grd_predict_train=grd.predict(Strain)

mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#run cross-validation on best hyperparameters
grd_cv_score = cross_val_score(grd, Strain, Y_Train, cv=8)

print(grd_cv_score)
print('\n')

print(" - gb: ",grd_cv_score.mean())

#test data
grd_predict_Test=grd.predict(Stest)

pred=pd.DataFrame(grd_predict_Test,columns=["Survived"])
print(pred.head())
output_col=pd.concat([testData["PassengerId"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/SCM516/Result_HY_GB_Stack.csv',index=None)

{'max_depth': 150, 'learning_rate': 0.2}
RMSE (training) for gradient bossting:  0.003835
[-0.27601952  0.07690402  0.13009001 -0.00989602 -0.19116599  0.30509707
 -0.12517375  0.05042419]


 - gb:  -0.00496749638883974
   Survived
0 -0.000002
1 -0.000007
2  0.000904
3 -0.000006
4  0.339533
   PassengerId  Survived
0          892 -0.000002
1          893 -0.000007
2          894  0.000904
3          895 -0.000006
4          896  0.339533
